# Summary
Here we do a validation of the 'simple' jaccard correlations by repeating some of the Spec2Vec metrics. Much of the code is directly copied from Florian's notebooks here https://github.com/iomega/spec2vec_gnps_data_analysis/blob/master/notebooks/

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import time

data_path = "/mnt/scratch/louwe015/Mass_differences/data/"

In [2]:
all_pos_version1 = False
if all_pos_version1:
    all_pos = "gnps_positive_ionmode_cleaned_by_matchms_and_lookups"
else:
    all_pos = "ALL_GNPS_210125_positive_cleaned_by_matchms_and_lookups"

print(all_pos)

ALL_GNPS_210125_positive_cleaned_by_matchms_and_lookups


In [3]:
if all_pos_version1:
    jacc_mat_file = None  # I didn't run this one
else:
    jacc_mat_file = "jaccard_matrix_allpos2_04-03-2021.npz"

jacc_mat_path = os.path.join(data_path, 'jaccard_matrices', jacc_mat_file)
print(jacc_mat_file, os.path.isfile(jacc_mat_path))

jaccard_matrix_allpos2_04-03-2021.npz True


## Load jaccard matrix and get MDs with correlations 0.1-0.5

In [41]:
low_cutoff = 0.1
high_cutoff = 0.5

In [6]:
jacc_mat_npz = np.load(jacc_mat_path, allow_pickle=True)

In [9]:
j_rows, j_cols, jacc_mat_sparse = [jacc_mat_npz[name] for name in jacc_mat_npz.files]
jacc_mat_sparse = jacc_mat_sparse.item()

In [27]:
jacc_mat_sparse

<61348x115913 sparse matrix of type '<class 'numpy.float64'>'
	with 384463253 stored elements in Compressed Sparse Row format>

In [34]:
maxes = jacc_mat_sparse.max(axis=1).toarray()

In [42]:
maxes_inds = [i for i, val in enumerate(maxes) if val[0] >= low_cutoff and val[0] <= high_cutoff]

In [45]:
maxes[:5], maxes_inds[:5]  # seems to have worked

(array([[0.13313639],
        [0.24642122],
        [0.1013701 ],
        [0.09151063],
        [0.08100498]]),
 [0, 1, 2, 5, 6])

In [54]:
# get MD names corresponding to these rows
chosen_mds = list(j_rows[maxes_inds])
j_rows[:10], chosen_mds[:5]

(array(['37.00', '37.01', '37.02', '37.03', '37.04', '37.05', '37.06',
        '37.07', '37.08', '37.09'], dtype='<U6'),
 ['37.00', '37.01', '37.02', '37.05', '37.06'])

In [53]:
# del(jacc_mat_sparse)
# del(jacc_mat_npz)

## Save/load chosen MDs

In [60]:
# save chosen MDs
out_indices = os.path.join(data_path, 'jaccard_matrices', jacc_mat_file.rpartition('.')[0]+"_chosen_MDs_0.1-0.5.txt")
print(out_indices)
with open(out_indices, 'w') as outf:
    for md in chosen_mds:
        outf.write(f'{md}\n')

/mnt/scratch/louwe015/Mass_differences/data/jaccard_matrices/jaccard_matrix_allpos2_04-03-2021_chosen_MDs_0.1-0.5.txt


In [9]:
out_indices = os.path.join(data_path, 'jaccard_matrices', jacc_mat_file.rpartition('.')[0]+"_chosen_MDs_0.1-0.5.txt")
print(out_indices, os.path.isfile(out_indices))
with open(out_indices, 'r') as inf:
    chosen_mds = [line.strip() for line in inf]
len(chosen_mds), chosen_mds[:5]

/mnt/scratch/louwe015/Mass_differences/data/jaccard_matrices/jaccard_matrix_allpos2_04-03-2021_chosen_MDs_0.1-0.5.txt True


(16595, ['37.00', '37.01', '37.02', '37.05', '37.06'])

## Load processed spectra for cosine, s2v, s2v+MDs
Spectra processed for s2v, s2v+MD, cosine scoring are loaded from notebook 1.

In [10]:
# load top30 file
top30_file = os.path.join(data_path, all_pos + "_top30_peaks.pickle")
if os.path.exists(top30_file):
    with open(top30_file, 'rb') as inf:
        spectrums_top30 = pickle.load(inf)  # list of matchms.Spectrum.Spectrum
else:
    print("error")
# load processed file
processed_file = os.path.join(data_path, all_pos + "_processed.pickle")
if os.path.exists(processed_file):
    with open(processed_file, 'rb') as inf:
        spectrums_processed = pickle.load(inf)  # list of matchms.Spectrum.Spectrum
else:
    print("error")
# load classical file
classical_file = os.path.join(data_path, all_pos + "_classical.pickle")
if os.path.exists(classical_file):
    with open(classical_file, 'rb') as inf:
        spectrums_classical = pickle.load(inf)  # list of matchms.Spectrum.Spectrum
else:
    print("error")

## Create test set of 1000 spectra whose inchikey exists in library >= 2 times

In [13]:
Inchikeys = []
for spec in spectrums_top30:
    Inchikeys.append(spec.get("inchikey"))
inchikeys_pd = pd.Series([x for x in Inchikeys if x])
inchikeys_pd.str[:14].value_counts()[:5]

NEGQHKSYEYVFTD    432
SULIDBRAXVDKBU    426
SRIGHEHXEGELQJ    338
IQGPMZRCLCCXAG    308
RWKUXQNLWDTSLO    235
dtype: int64

### Randomly select 1000 inchikeys that exist >=2 times in the dataset

In [16]:
min_copies_in_data = 2

suitable_inchikeys = pd.DataFrame(\
                                  inchikeys_pd.str[:14].value_counts()\
                                  [inchikeys_pd.str[:14].value_counts().values >= min_copies_in_data])
suitable_inchikeys.reset_index(level=suitable_inchikeys.index.names, inplace=True)
suitable_inchikeys.columns = (['inchikey14', 'occurences'])

# Important: sort values to make it reproducible (same occurences have random order otherwise!)
suitable_inchikeys = suitable_inchikeys.sort_values(['occurences', 'inchikey14'], ascending=False)

print(suitable_inchikeys.head(5))
suitable_inchikeys.tail(5)

       inchikey14  occurences
0  NEGQHKSYEYVFTD         432
1  SULIDBRAXVDKBU         426
2  SRIGHEHXEGELQJ         338
3  IQGPMZRCLCCXAG         308
4  RWKUXQNLWDTSLO         235


,inchikey14,occurences
7961,ABRULANJVVJLFI,2
7911,ABOPTWOWDLNBOE,2
8268,ABBPFXQJIWUCKF,2
8525,AAIIJIOWYALNCC,2
7631,AABZZWPMCAZHFC,2
